# MOwNiT - Rozwiązywanie równań nieliniowych

In [1]:
from typing import Callable
import numpy as np

## Metoda bisekcji

Metoda bisekcji opiera się na następującym twierdzeniu:

> Jeżeli $f$ jest funkcją ciągłą w przedziale domkniętym $[a,b]$ taką, że $f(a)\cdot f(b)<0$, to równanie $f(x)=0$ ma w przedziale $(a,b)$ co najmniej jedno rozwiązanie.

Zawuważmy, że powyższe twierdzenie tak naprawdę mówi, iż w przedziale $(a,b)$ funkcja $f$ ma nieparzystą liczbę rozwiązań.

Poniższy algortym poszukuje **jednego** przybliżonego miejsca zerowego zadanej funkcji *metodą bisekcji*, czyli podziału odcinka na dwie części. Na samym początku działania algorytmu (moment startu) bierzemy pod uwagę cały odcinek $[a,b]$ i dzielimy go dokładnie na pół - obliczamy $x_0=\frac{a+b}{2}$. Jeżeli $x_0$ jest miejscem zerowy, to algorytm się kończy - szukamy jednego, dowolnego miejsca zerowego, nie wszystkich. Jeżeli $x_0$ nie jest miejscem zerowym, to mamy dwie (nie wykluczające się) możliwości:
- w przedziale $(a,x_0)$ funkcja $f$ ma nieparzystą liczbę miejsc zerowych, tzn. funkcja $f$ ma na krańcach tego przedziału przeciwne znaki
- w przedziale $(x_0,b)$ funkcja $f$ ma nieparzystą liczbę miejsc zerowych, tzn. funkcja $f$ ma na krańcach tego przedziału przeciwne znaki

Któraś z powyższych możliwości musi zachodzić (możliwe, że obydwie). Bierzemy ten przedział, na krańcach którego funkcja ma przeciwne znaki i powtarzamy całą procedurę od początku.

Oczywiście może się zdarzyć, że nasz algorytm będzie zbieżny w nieskończoności, tzn. aby punkt $x_0$ faktycznie był miejscem zerowym musielibyśmy wykonać nieskończenie wiele kroków, zatem najczęściej poszukujemy miejsca zerowego z zadaną dokładnością: w momencie, gdy długość aktualnie badanego przedziału jest już mniejsza od zadanego na początku $\varepsilon$ oraz moduł wartości funkcji w połowie przedziału $|f(x_0)|$ również jest mniejszy od $\varepsilon$, to przyjmujemy $x_0$ za dostatecznie dobre przybliżenie miejsca zerowego i kończymy algorytm.

### Algorytm

1. Start: $x_p=a$, $x_k=b$.
2. Obliczamy $x_0=\frac{x_k+x_p}{2}$.
2. Sprawdzamy, czy $x_0$ jest szukanym miejscem zerowym lub jego przylbiżeniem z wystarczającą dokładnością. Jeżeli TAK -> Koniec. Jeżeli NIE, to sprawdzamy, czy $f(x_p)f(x_0)<0$:
    - jeżeli TAK, to $x_k=x_0$ i wracamy do punktu 2.
    - jeżeli NIE, to $x_p=x_0$ i wracamy do punktu 2.

### Ćwiczenie 1.

1. Napisz funkcję ```metoda_bisekcji```, która będzie szukać miejsc zerowych zadanej funkcji $f(x)$ na zadanym przedziale $[a,b]$ z zadaną dokładnością $e$. Napisz funkcję tak, aby przy niespełnionym założeniu o przeciwnych znakach na końcach przedziału funkcja wyświetlała komunikat o błędzie.
2. Sprawdź działanie funkcji na przykładach:
    - $f(x)=x^2-1$, $x\in[0,2]$
    - $f(x)=x^3-7x+6$, $x\in[-4,4]$
    - $f(x)=\ln x$, $x\in[0.5,2]$
    - $f(x)=x^2$, $x\in[-1,1]$

In [2]:
def bisection_method(fun: Callable[[np.float32], np.float32], a: np.float32, b: np.float32, eps: np.float32 = None, max_it: int = 100):
    if eps is None:
        eps = np.finfo(np.float32).eps * (a + b) / 2

    if not fun(a) * fun(b) < 0:
        raise Exception("Signs of f(a) and f(b) are not opposite!!!")

    for _ in range(max_it):
        x = (a + b) / 2

        f_x = fun(x)

        if abs(f_x) < eps:
            return x
        elif fun(a) * f_x < 0:
            b = x
        else:
            a = x

    return x

In [3]:
f1: Callable[[np.float32], np.float32] = lambda x: x ** 2 - 1
f2: Callable[[np.float32], np.float32] = lambda x: x ** 3 - 7 * x + 6
f3: Callable[[np.float32], np.float32] = np.log
f4: Callable[[np.float32], np.float32] = lambda x: x ** 2

In [4]:
a1, b1 = np.float32(0), np.float32(2)
x1 = bisection_method(f1, a1, b1)
print(x1)

1.0


In [5]:
a2, b2 = np.float32(-4), np.float32(4)
x2 = bisection_method(f2, a2, b2)
print(x2)

-2.0


In [6]:
a3, b3 = np.float32(0.5), np.float32(2)
x3 = bisection_method(f3, a3, b3)
print(x3)

0.9999999


In [7]:
a4, b4 = np.float32(-1), np.float32(1)
try:
    x4 = bisection_method(f4, a4, b4)
    print(x4)
except Exception as error:
    print(error)

Signs of f(a) and f(b) are not opposite!!!


## Metoda Newtona (metoda stycznych)

*Metoda Newtona* jest kolejnym algorytmem poszukiwania miejsc zerowych funkcji. Metoda przyjmuje następujące założenia:

> 1. Funkcja $f$ jest funkcją ciągłą i różniczkowalną na przedziale $[a,b]$.
> 2. Pierwsza i druga pochodna funkcji $f$ mają stałe znaki na przedziale $[a,b]$.
> 2. $f(a)\cdot f(b)<0$
> 2. W przedziale $[a,b]$ funkcja $f$ ma dokładnie jeden pierwiastek.

Zaczynamy od punktu będącego początkiem przedziału. Następnie wyznaczamy styczną do wykresu funkcji w puncie $(a,f(a))$ i obliczamy współrzędne $(x,0)$ jej punktu przecięcia z osią OX. Jeżeli punkt $x$ jest szukanym miejscem zerowym (lub jego odpowiednim przybliżeniem), to kończymy algorytm. W przeciwnym wypadku, dzięki stałej monotoniczności i wypukłości funkcji $f$ wiemy, że szukane miejsce zerowe znajduje się gdzieś w przedziale $(x,b]$ i powtarzamy rozumowanie.

### Algorytm

1. Start: $x_n=a$.
2. Obliczamy współrzędne $(x_{n+1},0)$ punktu przecięcia stycznej do funkcji w punkcie $(x_n,f(x_n))$ z osią OX.
3. Sprawdzamy czy $x_{n+1}$ jest miejscem zerowym lub jego odpowiednim przybliżeniem. Jeżeli TAK -> Koniec. Jeżeli NIE, to uaktualniamy $x_n=x_{n+1}$ i wracamy do punktu 2.

### Ćwiczenie 2.

1. Napisz funkcję ```metoda_Newtona```, która będzie szukać miejsc zerowych zadanej funkcji $f(x)$ na zadanym przedziale $[a,b]$ z zadaną dokładnością $e$.
2. Sprawdź działanie tej funkcji na następujących przykładach:
    - $f(x)=\ln x$, $x\in[0.5,1.5]$
    - $f(x)=x^2-1$, $x\in[0.5,2]$
3. Dlaczego algorytm nie będzie działał dla $f(x)=x^2-1$, $x\in[0,2]$?

    -  Odpowidź: Algorytm nie będzie działać, ponieważ w pierwszej iteracji algorytmu będziemy mieli dzielienie przez wartość pochodnej z funkcji f w punkcjie 0, która jest równa: $\frac{df}{dx} (0) = 2 \cdot 0 = 0$, a dzielenie przez zero daje nam wartość ```nan``` naszego nowego punktu x.

4. Za pomocą funkcji ```metoda_Newtona``` stwórz własną funkcję, która będzie obliczać pierwiastek kwadratowy z zadanej liczby nieujemnej (bez użycia funkcji ```sqrt``` z różnych bibliotek i bez użycia potęgowania).

In [8]:
def newton_method(fun: Callable[[np.float32], np.float32], df: Callable[[np.float32], np.float32], a: np.float32, b: np.float32, eps: np.float32 = None, max_it: int = 100):
    if eps is None:
        eps = np.finfo(np.float32).eps * (a + b) / 2

    if not fun(a) * fun(b) < 0:
        raise Exception("Signs of f(a) and f(b) are not opposite!!!")

    x = a
    for _ in range(max_it):
        if abs(fun(x)) < eps:
            break

        x -= fun(x) / df(x)


    return x

In [9]:
a5, b5 = np.float32(0.5), np.float32(1.5)
try:
    x5 = newton_method(f3, lambda x: 1 / x, a5, b5)
    print(x5)
except Exception as error:
    print(error)

1.0


In [10]:
a6, b6 = np.float32(0.5), np.float32(2)
try:
    x6 = newton_method(f1, lambda x: 2 * x, a6, b6)
    print(x6)
except Exception as error:
    print(error)

1.0


In [11]:
def my_sqrt(a: np.float32):
    if a < 1:
        start = a
        end = 1
    else:
        start = 1e-6
        end = a + 1

    return newton_method(lambda x: x ** 2 - a, lambda x: 2 * x, start, end, eps = 1e-10)

In [12]:
print(my_sqrt(4))
print(my_sqrt(3))
print(my_sqrt(1))
print(my_sqrt(0.1))
print(my_sqrt(1e8))

2.0000000000000107
1.7320508075688774
1.0000000000000053
0.31622776601683794
10000.0


## Metoda Eulera (metoda siecznych)

Założenia metody siecznych:

> 1. Funkcja $f$ jest funkcją ciągłą na przedziale $[a,b]$.
> 2. $f(a)\cdot f(b)<0$
> 2. W przedziale $[a,b]$ funkcja $f$ ma dokładnie jeden pierwiastek.

Metoda siecznych wykorzystuje fakt, że dla odpowiednio małego przedziału $[x_1,x_2]$ sieczna przechodząca przez punkty $(x_1,f(x_1))$, $(x_2,f(x_2))$ jest wystarczająco dobrą aproksymacją funkcji $f$ na $[x_1,x_2]$. To podejście ma jedną podstawową wadę: jeżeli wyjściowy przedział $[a,b]$ jest zbyt duży, algorytm może nie zadziałać, jak powinien.

### Algorytm

1. Start: $x_1=a$, $x_2=b$.
2. Wyznaczamy punkt $(x_0,0)$ przecięcia siecznej przechodzącej przez punkty $(x_1,f(x_1))$, $(x_2,f(x_2))$ z osią OX.
3. Sprawdzamy czy $x_0$ jest miejscem zerowym lub jego odpowiednim przybliżeniem. Jeżeli TAK -> Koniec. Jeżeli NIE, to:
    - $x_1=x_2$
    - $x_2=x_0$
    - wracamy do punktu 2.

### Ćwiczenie 3.

1. Napisz funkcję ```metoda_siecznych```, która będzie szukać miejsc zerowych zadanej funkcji $f(x)$ na zadanym przedziale $[a,b]$ z zadaną dokładnością $e$.
2. Sprawdź działanie tej funkcji na następujących przykładach:
    - $f(x)=\sqrt[3]{x}$, $x\in[-1,1]$
    - $f(x)=x^3-7x+6$, $x\in[-4,-2]$
    - $f(x)=\text{arctg}\, x$, $x\in[-1,3]$
3. Dlaczego algorytm nie będzie działał dla $f(x)=x^3-7x+6$, $x\in[0,3]$?

    - Odpowiedź: Algorytm nie będzie działał w tym przedziale, ponieważ nie są spełnione założenia 2. oraz 3.

4. Sprawdź, czy algorytm zadziała poprawnie dla $f(x)=x^2-4$, $x\in[-1,10]$.

In [13]:
def secant_method(fun: Callable[[np.float32], np.float32], a: np.float32, b: np.float32, eps: np.float32 = None, max_it: int = 100):
    if eps is None:
        eps = np.finfo(np.float32).eps * (a + b) / 2

    if not fun(a) * fun(b) < 0:
        raise Exception("Signs of f(a) and f(b) are not opposite!!!")

    x = a
    for _ in range(max_it):
        if abs(fun(x)) < eps:
            break

        m = (fun(b) - fun(a)) / (b - a)
        x -= fun(x) / m

    return x


In [14]:
f7: Callable[[np.float32], np.float32] = np.cbrt
f8: Callable[[np.float32], np.float32] = lambda x: x ** 3 - 7 * x + 6
f9: Callable[[np.float32], np.float32] = np.arctan

In [15]:
a7, b7 = np.float32(-1), np.float32(1)
try:
    x7 = secant_method(f7, a7, b7)
    print(x7)
except Exception as error:
    print(error)

0.0


In [16]:
a8, b8 = np.float32(-4), np.float32(-2)
try:
    x8 = secant_method(f8, a8, b8)
    print(x8)
except Exception as error:
    print(error)

-3.0


In [17]:
a9, b9 = np.float32(-1), np.float32(3)
try:
    x9 = secant_method(f9, a9, b9)
    print(x9)
except Exception as error:
    print(error)

-0.0066939443


## Regula falsi

Założenia metody *Regula falsi* są takie same, jak założenia metody Newtona:

> 1. Funkcja $f$ jest funkcją ciągłą i różniczkowalną na przedziale $[a,b]$.
> 2. Pierwsza i druga pochodna funkcji $f$ mają stałe znaki na przedziale $[a,b]$.
> 2. $f(a)\cdot f(b)<0$
> 2. W przedziale $[a,b]$ funkcja $f$ ma dokładnie jeden pierwiastek.

Metoda jest bardzo podobna do metody siecznych z tą różnicą, że zmienia się tylko jeden z punktów, przez który przeprowadzne są kolejne sieczne. Drugi punkt jest ustalony i jest to jeden z końców odcinka.

### Algorytm

1. Start: $x_1=a$, $x_2=b$.
2. Wyznaczamy punkt $(x_0,0)$ przecięcia siecznej przechodzącej przez punkty $(x_1,f(x_1))$, $(x_2,f(x_2))$ z osią OX.
3. Sprawdzamy czy $x_0$ jest miejscem zerowym lub jego odpowiednim przybliżeniem. Jeżeli TAK -> Koniec. Jeżeli NIE, to:
    - jeżeli $f(x_1)f(x_0)<0$, to $x_2=x_0$, wartość $x_1$ się nie zmienia
    - jeżeli $f(x_2)f(x_0)<0$, to $x_1=x_0$, wartość $x_2$ się nie zmienia
    - wracamy do punktu 2.
    
Dzięki założeniom funkcja $f$ jest monotoniczna w $[a,b]$, ma również określoną wypukłość, zatem w punkcie 3 algorytmu zawsze będzie wybierana ta sama opcja i przez cały algorytm jedna z wartości $x_1=a$ albo $x_2=b$ pozostanie bez zmian. Można na samym początku algorytmu ustalić, która z tych wartości pozostanie stała i zrezygnować z dwóch instrukcji warunkowych w pętli, co znacznie przyspiesza obliczenia, jednak wtedy algorytm jest mniej uniwersalny.

### Ćwiczenie 4.

1. Napisz funkcję ```regula_falsi```, która będzie szukać miejsc zerowych zadanej funkcji $f(x)$ na zadanym przedziale $[a,b]$ z zadaną dokładnością $e$.
2. Sprawdź działanie tej funkcji na przykładach:
    - $f(x)=e^x-1$, $x\in[-1,1]$
    - $f(x)=x^2-1\, x$, $x\in[0,2]$
4. Sprawdź, czy algorytm zadziała poprawnie w przypadku funkcji:
    - $f(x)=x^2-4$, $x\in[-1,10]$ (funkcja nie jest monotoniczna, ale ma stałą wypukłość)
    - $f(x)=\sqrt[3]{x}$, $x\in[-3,3]$ (funkcja ma punkt przegięcia, ale jest monotoniczna)
    - $f(x)=\sin x$, $x\in[-2,2]$ (funkcja ma punkt przegięcia i nie jest monotoniczna)

In [18]:
def falsi_method(fun: Callable[[np.float32], np.float32], a: np.float32, b: np.float32, eps: np.float32 = None, max_it: int = 100):
    if eps is None:
        eps = np.finfo(np.float32).eps * (a + b) / 2

    if not fun(a) * fun(b) < 0:
        raise Exception("Signs of f(a) and f(b) are not opposite!!!")

    x = a
    for _ in range(max_it):
        if abs(fun(x)) < eps:
            break

        x = b - (fun(b) * (a - b)) / (fun(a) - fun(b))
        if fun(x) * fun(a) < 0:
            b = x
        else:
            a = x

    return x

In [19]:
f10: Callable[[np.float32], np.float32] = lambda x: np.exp(x) - 1
f11: Callable[[np.float32], np.float32] = lambda x: x ** 2 - 1

In [20]:
a10, b10 = np.float32(-1), np.float32(3)
try:
    x10 = secant_method(f10, a10, b10)
    print(x10)
except Exception as error:
    print(error)

-8.474606e-08


In [21]:
a11, b11 = np.float32(0), np.float32(2)
try:
    x11 = secant_method(f11, a11, b11)
    print(x11)
except Exception as error:
    print(error)

1.0


In [22]:
f12: Callable[[np.float32], np.float32] = lambda x: x ** 2 - 4
f13: Callable[[np.float32], np.float32] = np.cbrt
f14: Callable[[np.float32], np.float32] = np.sin

In [23]:
a12, b12 = np.float32(-1), np.float32(10)
try:
    x12 = secant_method(f12, a12, b12)
    print(x12)
except Exception as error:
    print(error)

1.9999999


In [24]:
a13, b13 = np.float32(-3), np.float32(3)
try:
    x13 = secant_method(f13, a13, b13)
    print(x13)
except Exception as error:
    print(error)

0.0


In [25]:
a14, b14 = np.float32(-2), np.float32(2)
try:
    x14 = secant_method(f14, a14, b14)
    print(x14)
except Exception as error:
    print(error)

0.0


Jak widać Reguła Falsi działa również w pokazanych przypadkach, pomimo tego, że nie spełniają założeń algorytmu. Pokazuje to, że założenia zapewniają nas, że pierwiastek zostanie znaleziony, natomiast nie oznaczają, iż algorytm nie będzie działał poprawnie w przypadku niespełnienia tych założeń.

### Ćwiczenie 5.

W pakiecie `scipy.optimize` znajdują się dwie funkcje do znajdowania pierwiastków: `fsolve()` oraz `root()`. Wykorzystując dokumentację biblioteki `scipy` za pomocą odpowiednio dobranej funkcji rozwiąż te przykłady z ćwiczeń 1-4, których nie udało się rozwiązać podstawowymi algorytmami.

In [26]:
from scipy.optimize import root

Znajdźmy pierwiastki funkcji $ f(x) = x^2 $ wykorzystując funkcję ```root```. W przedziale $<-1,1>$

In [27]:
sol = root(f4, x0 = 1)
print(sol.x.item(0))

5.681053833367079e-84
